# Playground

Like the name states, a notebook to play around with code based on learning from other notebooks

In [24]:
import os
from langchain.prompts  import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain, SequentialChain
from langchain.callbacks import get_openai_callback
import tiktoken

Useful functions below (e.g. get a sense of token usage)

In [2]:
# Functions
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [3]:
# Make sure you have setup your environmental variables beforehand
os.environ["OPENAI_API_KEY"]
os.environ["OPENAI_ORGANIZATION"]

'org-HCpyLwy1gf2wzE49sCGQRU79'

In [4]:
llm_openai = OpenAI(model_name = "gpt-4-0613", temperature=0)


/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain/llms/openai.py:785: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


#### Let's play around first with our prompt engineering

We'll be testing use of a prompt that requires answers to be based on quotes from One Piece. <b>Make sure to clean any pasted text prior to feeding into model to avoid errors </b>

Quotes source at: https://everydaypower.com/one-piece-quotes/

In [5]:
op_quotes = """
"I've been picking fights all along!" - Luffy
"You are too from that generation." - Luffy
"Are we friends? Or are we foes? That kind of thing you decide yourselves!" - Luffy
“I don't wanna conquer anything. It's just that the person with the most freedom on the sea is the pirate king.” - Luffy
“If you don't take risks you can't create a future.” - Luffy
“Power isn't determined by your size, but by the size of your heart and dreams.” - Luffy
“Being alone is more painful than getting hurt.” - Luffy
“It's not about whether it's impossible or not, I'm doing it because I want to.” - Luffy
"Life is like a pencil that will surely run out, but will leave the beautiful writing of life." - Nami
"Everyone's weak when they hesitate." - Nami
"Everyone is expecting help from you, but you come here and don't think about doing anything to help those people?" - Nami
“I'm the smartest member in the Straw Hat group.” - Nami
“If you think I'm just another cute girl, you're dead wrong!” - Nami
“I'll have mapped out the entire world!” - Nami
“Just one more time and everything will be the same. Everything will come back to me. And I will be free from Arlong. Bellemere-san. Finally, I can laugh with all my heart.” - Nami
"""

op_prompt = """You are an AI assistant that provides life advice based on the quotes below. 
The name after the dash is the author. Please reference quotes in answers. Include the author name. 
If you cannot answer based on the quotes, simply state that you don't know."

Quotes:
{quotes}

Question: {query}

Answer: """

In [6]:
op_prompt_template = PromptTemplate(
    input_variables=["query", "quotes"],
    template=op_prompt
)

Test that the prompt works. <b>Notice how we adjust the prompt to take in multiple input variables</b>

In [7]:
print(op_prompt_template.format(query = "What should we do when we lose?", quotes=op_quotes))

You are an AI assistant that provides life advice based on the quotes below. 
The name after the dash is the author. Please reference quotes in answers. Include the author name. 
If you cannot answer based on the quotes, simply state that you don't know."

Quotes:

"I've been picking fights all along!" - Luffy
"You are too from that generation." - Luffy
"Are we friends? Or are we foes? That kind of thing you decide yourselves!" - Luffy
“I don't wanna conquer anything. It's just that the person with the most freedom on the sea is the pirate king.” - Luffy
“If you don't take risks you can't create a future.” - Luffy
“Power isn't determined by your size, but by the size of your heart and dreams.” - Luffy
“Being alone is more painful than getting hurt.” - Luffy
“It's not about whether it's impossible or not, I'm doing it because I want to.” - Luffy
"Life is like a pencil that will surely run out, but will leave the beautiful writing of life." - Nami
"Everyone's weak when they hesitate." - 

#### Knowing token needed per prompt is important, these functions will help us keep track of that

Ensure we are within the token limit

In [8]:
print("Number of words: {}".format(
    len(op_prompt_template.format(query = "What should we do when we lose?", quotes=op_quotes))
    ))

n_tokens = num_tokens_from_string(
    string=op_prompt_template.format(query = "What should we do when we lose?", quotes=op_quotes),
    encoding_name="cl100k_base"
)

print("Number of tokens: {}".format(n_tokens))

Number of words: 1536
Number of tokens: 381


Feed a negative control ensuring LLM is not supposed what is not supposed to do according to prompt instructions

In [9]:
llm_openai(op_prompt_template.format(query = "What is Doge?", quotes=op_quotes))

"I'm sorry, but I cannot provide an answer based on the quotes provided."

Now feed it actual prompt of interest

In [10]:
print(
    llm_openai(
        op_prompt_template.format(query = "What should we do around weak people?", quotes=op_quotes)
        )
    )

Based on the quotes, it's important to remember that "Power isn't determined by your size, but by the size of your heart and dreams." - Luffy. This means that even if someone appears weak, they may have a strong heart and big dreams. It's also important to remember that "Everyone's weak when they hesitate." - Nami. This suggests that weakness can be temporary and is often related to uncertainty or fear. Therefore, around weak people, we should be supportive, encourage them to follow their dreams, and help them overcome their fears and uncertainties.


#### Now let's go ahead and combine prompt engineering with chains

We are printing a traditional prompt below to remind you what is doing. Recall that the original prompt does not contain the query and that this is just a placeholder which we replace with the actual user input

In [11]:
print(op_prompt_template.format(query="What is life?", quotes=op_quotes))

You are an AI assistant that provides life advice based on the quotes below. 
The name after the dash is the author. Please reference quotes in answers. Include the author name. 
If you cannot answer based on the quotes, simply state that you don't know."

Quotes:

"I've been picking fights all along!" - Luffy
"You are too from that generation." - Luffy
"Are we friends? Or are we foes? That kind of thing you decide yourselves!" - Luffy
“I don't wanna conquer anything. It's just that the person with the most freedom on the sea is the pirate king.” - Luffy
“If you don't take risks you can't create a future.” - Luffy
“Power isn't determined by your size, but by the size of your heart and dreams.” - Luffy
“Being alone is more painful than getting hurt.” - Luffy
“It's not about whether it's impossible or not, I'm doing it because I want to.” - Luffy
"Life is like a pencil that will surely run out, but will leave the beautiful writing of life." - Nami
"Everyone's weak when they hesitate." - 

We are going to re-design the prompt around the important piece of info: the quotes. The template will contain instructions to output specific information. The template will not contain the information where to draw this info from, so it will have the input query blank to later enter the quotes of interest. This will be our <b>personality fact extraction prompt template</b>

Recall that the previous prompt had specific instructions on what it was instructed to (ie. life advice AI) and was required to provide a specific type of answer (ie. ensure to use a quote of the list + author in answer). Therefore, it cannot be used here. <u>Make sure to understand the difference between what each prompt is intended to do</u>

In [19]:
rank_template = """
Each quote below lists its author right after a dash. Based on each author's quotes, provide 3 key facts about each of them.
Don't provide the actual quotes in the answer.
Give each fact a number:\n{quotes}
"""

personality_facts_prompt = PromptTemplate(input_variables=["quotes"],
                                          template=rank_template)

print(personality_facts_prompt.format(quotes=op_quotes))


Each quote below lists its author right after a dash. Based on each author's quotes, provide 3 key facts about each of them.
Don't provide the actual quotes in the answer.
Give each fact a number:

"I've been picking fights all along!" - Luffy
"You are too from that generation." - Luffy
"Are we friends? Or are we foes? That kind of thing you decide yourselves!" - Luffy
“I don't wanna conquer anything. It's just that the person with the most freedom on the sea is the pirate king.” - Luffy
“If you don't take risks you can't create a future.” - Luffy
“Power isn't determined by your size, but by the size of your heart and dreams.” - Luffy
“Being alone is more painful than getting hurt.” - Luffy
“It's not about whether it's impossible or not, I'm doing it because I want to.” - Luffy
"Life is like a pencil that will surely run out, but will leave the beautiful writing of life." - Nami
"Everyone's weak when they hesitate." - Nami
"Everyone is expecting help from you, but you come here and do

In [20]:
n_tokens = num_tokens_from_string(
    string=personality_facts_prompt.format(quotes=op_quotes),
    encoding_name="cl100k_base"
)
print("Number of tokens in prompt: {}".format(n_tokens))

Number of tokens in prompt: 357


Now that we have the personality fact prompt, let's run it

In [21]:
op_fact_personality_chain = LLMChain(llm=llm_openai, prompt=personality_facts_prompt)
op_fact_personality_chain

LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['quotes'], output_parser=None, partial_variables={}, template="\nEach quote below lists its author right after a dash. Based on each author's quotes, provide 3 key facts about each of them.\nDon't provide the actual quotes in the answer.\nGive each fact a number:\n{quotes}\n", template_format='f-string', validate_template=True), llm=OpenAIChat(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-4-0613', model_kwargs={'temperature': 0}, openai_api_key=None, openai_api_base=None, openai_proxy=None, max_retries=6, prefix_messages=[], streaming=False, allowed_special=set(), disallowed_special='all'), output_key='text', output_parser=StrOutputParser(), return_final_only=True, llm_kwargs={})

In [22]:
print(op_fact_personality_chain.run(op_quotes))

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).


Luffy:
1. Luffy is a character who is not afraid of conflict, often initiating fights himself.
2. He values freedom above all else, aspiring to be the pirate king, the person with the most freedom on the sea.
3. Luffy believes in the power of the heart and dreams, and is willing to take risks to create a future. He also acknowledges the pain of loneliness.

Nami:
1. Nami is a character who values life and sees it as a beautiful writing that will eventually run out.
2. She considers herself the smartest member of the Straw Hat group and is determined to map out the entire world.
3. Nami is a strong and independent character who doesn't want to be seen as just another cute girl. She also shows a desire for freedom and happiness, as seen in her wish to be free from Arlong and to laugh with all her heart.


#### Now to the fun part, let's build an actual chain that takes as input quotes, generates personality traits from such quotes and then predicts how a person may respond in varying situtations

First we need to generate the prompt template chain portion that extracts the person's traits based on their name <b>Find out if there is a less costly way of doing this</b>

In [41]:
personality_selection_template = """
Use the following name: {non_ai_person} and extract their corresponding numbered list from the paragraph below. Don't deviate from the text.
If you can't identify the person, answer with: "I don't know"\n{people_facts}
"""

personality_selection_prompt = PromptTemplate(
    input_variables=["people_facts", "non_ai_person"],
    template=personality_selection_template
)

Then convert it into a chain (Don't forget that the sequential chain needs chains, not prompts)

In [42]:
personality_selection_chain = LLMChain(llm=llm_openai, prompt=personality_selection_prompt)
personality_selection_chain

LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['people_facts', 'non_ai_person'], output_parser=None, partial_variables={}, template='\nUse the following name: {non_ai_person} and extract their corresponding numbered list from the paragraph below. Don\'t deviate from the text.\nIf you can\'t identify the person, answer with: "I don\'t know"\n{people_facts}\n', template_format='f-string', validate_template=True), llm=OpenAIChat(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-4-0613', model_kwargs={'temperature': 0}, openai_api_key=None, openai_api_base=None, openai_proxy=None, max_retries=6, prefix_messages=[], streaming=False, allowed_special=set(), disallowed_special='all'), output_key='text', output_parser=StrOutputParser(), return_final_only=True, llm_kwargs={})

Now chain it with the previous chain that extract personality facts from quotes

In [43]:
personality_extraction_chain = SimpleSequentialChain(
    chains=[op_fact_personality_chain, personality_selection_chain],
    verbose=True
)

# personality_response = personality_extraction_chain.run(quotes=op_quotes, non_ai_person="Doge")
personality_response = personality_extraction_chain({"input":op_quotes, "non_ai_person":"Nami"})
personality_response

ValidationError: 1 validation error for SimpleSequentialChain
__root__
  Chains used in SimplePipeline should all have one input, got memory=None callbacks=None callback_manager=None verbose=False tags=None metadata=None prompt=PromptTemplate(input_variables=['people_facts', 'non_ai_person'], output_parser=None, partial_variables={}, template='\nUse the following name: {non_ai_person} and extract their corresponding numbered list from the paragraph below. Don\'t deviate from the text.\nIf you can\'t identify the person, answer with: "I don\'t know"\n{people_facts}\n', template_format='f-string', validate_template=True) llm=OpenAIChat(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-4-0613', model_kwargs={'temperature': 0}, openai_api_key=None, openai_api_base=None, openai_proxy=None, max_retries=6, prefix_messages=[], streaming=False, allowed_special=set(), d

Notice the error, SimpleSequentialChain expects one input per one output it produces. We need to modify these chain calls by including <b>the output_key argument</b> and rather use the <b>SequentialChain</b> function which allows for multiple input/output calls

In [56]:
op_fact_personality_chain = LLMChain(
    llm=llm_openai, 
    prompt=personality_facts_prompt, # has the "quotes" input variable
    output_key="people_facts",
    verbose=True
    )

personality_selection_chain = LLMChain(
    llm=llm_openai, 
    prompt=personality_selection_prompt, # has both "people_facts" and "non_ai_person" as input variables
    output_key="person_facts",
    verbose=True
    )

personality_extraction_chain = SequentialChain(
    input_variables=["quotes", "non_ai_person"],
    output_variables=["people_facts", "person_facts"],
    chains=[op_fact_personality_chain, personality_selection_chain],
    verbose=True
)

personality_response = personality_extraction_chain({"quotes":op_quotes, "non_ai_person":"Nami"})
personality_response




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Each quote below lists its author right after a dash. Based on each author's quotes, provide 3 key facts about each of them.
Don't provide the actual quotes in the answer.
Give each fact a number:

"I've been picking fights all along!" - Luffy
"You are too from that generation." - Luffy
"Are we friends? Or are we foes? That kind of thing you decide yourselves!" - Luffy
“I don't wanna conquer anything. It's just that the person with the most freedom on the sea is the pirate king.” - Luffy
“If you don't take risks you can't create a future.” - Luffy
“Power isn't determined by your size, but by the size of your heart and dreams.” - Luffy
“Being alone is more painful than getting hurt.” - Luffy
“It's not about whether it's impossible or not, I'm doing it because I want to.” - Luffy
"Life is like a pencil that will surely run out, but will leave the beautiful writing of life." - Nami
"Ever

{'quotes': '\n"I\'ve been picking fights all along!" - Luffy\n"You are too from that generation." - Luffy\n"Are we friends? Or are we foes? That kind of thing you decide yourselves!" - Luffy\n“I don\'t wanna conquer anything. It\'s just that the person with the most freedom on the sea is the pirate king.” - Luffy\n“If you don\'t take risks you can\'t create a future.” - Luffy\n“Power isn\'t determined by your size, but by the size of your heart and dreams.” - Luffy\n“Being alone is more painful than getting hurt.” - Luffy\n“It\'s not about whether it\'s impossible or not, I\'m doing it because I want to.” - Luffy\n"Life is like a pencil that will surely run out, but will leave the beautiful writing of life." - Nami\n"Everyone\'s weak when they hesitate." - Nami\n"Everyone is expecting help from you, but you come here and don\'t think about doing anything to help those people?" - Nami\n“I\'m the smartest member in the Straw Hat group.” - Nami\n“If you think I\'m just another cute girl, 

We got our desired response. Notice how we are able to store the mid-chain outputs as well in the dictionary object

In [57]:
print(personality_response["person_facts"])

Nami:
1. Nami is a character who values life and sees it as a beautiful writing that will eventually run out.
2. She considers herself the smartest member of the Straw Hat group and is determined to map out the entire world.
3. Nami is a strong and independent character who doesn't like being underestimated because of her appearance. She also shows a desire for freedom and happiness, as seen in her wish to be free from Arlong and to laugh wholeheartedly.


In [59]:
print(personality_response.keys())

dict_keys(['quotes', 'non_ai_person', 'people_facts', 'person_facts'])


Do a negative test to ensure it is working as expected. Input a person's name not found in the personality list

In [60]:
personality_response = personality_extraction_chain({"quotes":op_quotes, "non_ai_person":"Doge"})
personality_response


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Each quote below lists its author right after a dash. Based on each author's quotes, provide 3 key facts about each of them.
Don't provide the actual quotes in the answer.
Give each fact a number:

"I've been picking fights all along!" - Luffy
"You are too from that generation." - Luffy
"Are we friends? Or are we foes? That kind of thing you decide yourselves!" - Luffy
“I don't wanna conquer anything. It's just that the person with the most freedom on the sea is the pirate king.” - Luffy
“If you don't take risks you can't create a future.” - Luffy
“Power isn't determined by your size, but by the size of your heart and dreams.” - Luffy
“Being alone is more painful than getting hurt.” - Luffy
“It's not about whether it's impossible or not, I'm doing it because I want to.” - Luffy
"Life is like a pencil that will surely run out, but will leave the beautiful writing of life." - Nami
"Ever

{'quotes': '\n"I\'ve been picking fights all along!" - Luffy\n"You are too from that generation." - Luffy\n"Are we friends? Or are we foes? That kind of thing you decide yourselves!" - Luffy\n“I don\'t wanna conquer anything. It\'s just that the person with the most freedom on the sea is the pirate king.” - Luffy\n“If you don\'t take risks you can\'t create a future.” - Luffy\n“Power isn\'t determined by your size, but by the size of your heart and dreams.” - Luffy\n“Being alone is more painful than getting hurt.” - Luffy\n“It\'s not about whether it\'s impossible or not, I\'m doing it because I want to.” - Luffy\n"Life is like a pencil that will surely run out, but will leave the beautiful writing of life." - Nami\n"Everyone\'s weak when they hesitate." - Nami\n"Everyone is expecting help from you, but you come here and don\'t think about doing anything to help those people?" - Nami\n“I\'m the smartest member in the Straw Hat group.” - Nami\n“If you think I\'m just another cute girl, 

As expected and to proof it is specific to our personality traits, the final output yields a negative response

In [61]:
print(personality_response["person_facts"])

I don't know


Let's take the exercise one last step further an request to provide a <b>generated AI person response's to a situational environment</b>

Generate the additional LLM prompt and single chain below. We are copying the chains from above to keep tracks of variables used

In [ ]:
op_fact_personality_chain = LLMChain(
    llm=llm_openai, 
    prompt=personality_facts_prompt, # has the "quotes" input variable
    output_key="people_facts",
    verbose=True
    )

personality_selection_chain = LLMChain(
    llm=llm_openai, 
    prompt=personality_selection_prompt, # has both "people_facts" and "non_ai_person" as input variables
    output_key="person_facts",
    verbose=True
    )

In [68]:
situational_response_template = """
You are an AI whose aim is to emulate how a person will respond to real-life scenarios. 
You will only use the personality traits listed below to answer how this person would react under the scenario presented at the end.

Traits: {person_facts}\n\n

Scenario: {simulated_scenario}\n
"""

situational_response_prompt = PromptTemplate(
    input_variables= ["person_facts", "simulated_scenario"],
    template=situational_response_template
)

situation_response_chain = LLMChain(
    llm=llm_openai,
    prompt=situational_response_prompt,
    output_key="gen_ai_situational_response",
    verbose=True
)

Now incorporate it into full chain. <u>Note:</u> Keep track of variable names. Notice how output variables include all those generated throughout the process, and that they are not placed as "input_variables" even if they do enter a mid-chain. This is important to keep in mind

In [71]:
gen_ai_situational_response_chain = SequentialChain(
    input_variables=["quotes", "non_ai_person", "simulated_scenario"],
    output_variables=["people_facts", "person_facts", "gen_ai_situational_response"],
    chains=[op_fact_personality_chain, personality_selection_chain, situation_response_chain],
    verbose=True
)

simulated_scenario ="""
Your best friend comes to you devasted by the loss of their dog. What would you say to them to cheer them up? 
Speak from a first person point of view
"""

gen_ai_situational_response = gen_ai_situational_response_chain({
    "quotes":op_quotes, 
    "non_ai_person":"Nami",
    "simulated_scenario": simulated_scenario
    })



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Each quote below lists its author right after a dash. Based on each author's quotes, provide 3 key facts about each of them.
Don't provide the actual quotes in the answer.
Give each fact a number:

"I've been picking fights all along!" - Luffy
"You are too from that generation." - Luffy
"Are we friends? Or are we foes? That kind of thing you decide yourselves!" - Luffy
“I don't wanna conquer anything. It's just that the person with the most freedom on the sea is the pirate king.” - Luffy
“If you don't take risks you can't create a future.” - Luffy
“Power isn't determined by your size, but by the size of your heart and dreams.” - Luffy
“Being alone is more painful than getting hurt.” - Luffy
“It's not about whether it's impossible or not, I'm doing it because I want to.” - Luffy
"Life is like a pencil that will surely run out, but will leave the beautiful writing of life." - Nami
"Ever

{'quotes': '\n"I\'ve been picking fights all along!" - Luffy\n"You are too from that generation." - Luffy\n"Are we friends? Or are we foes? That kind of thing you decide yourselves!" - Luffy\n“I don\'t wanna conquer anything. It\'s just that the person with the most freedom on the sea is the pirate king.” - Luffy\n“If you don\'t take risks you can\'t create a future.” - Luffy\n“Power isn\'t determined by your size, but by the size of your heart and dreams.” - Luffy\n“Being alone is more painful than getting hurt.” - Luffy\n“It\'s not about whether it\'s impossible or not, I\'m doing it because I want to.” - Luffy\n"Life is like a pencil that will surely run out, but will leave the beautiful writing of life." - Nami\n"Everyone\'s weak when they hesitate." - Nami\n"Everyone is expecting help from you, but you come here and don\'t think about doing anything to help those people?" - Nami\n“I\'m the smartest member in the Straw Hat group.” - Nami\n“If you think I\'m just another cute girl, 

In [73]:
print(gen_ai_situational_response.keys())
print(gen_ai_situational_response["gen_ai_situational_response"])

dict_keys(['quotes', 'non_ai_person', 'simulated_scenario', 'people_facts', 'person_facts', 'gen_ai_situational_response'])
I'm really sorry to hear about your loss. I know how much you loved your dog. Remember, life is like a beautiful story, and just like every story, it has an end. Your dog's story was filled with love, joy, and companionship, and that's something to cherish. It's okay to feel sad, it's okay to grieve. But don't forget to celebrate the wonderful moments you shared together. Those memories are a part of your world map, and they will always be there. And remember, I'm here for you. Don't underestimate the power of friendship and shared sorrow. We'll get through this together.


The answer is beautifully written and you can see how it is reflective of what the character would say based on their traits. Pretty dope if you ask me.

### To summarize we developed the Gen AI Chain:

Quotes from various authors -> Summarizing key facts making up an author's personality -> Select a list of traits from only one author -> Present the author with a simulated scenario -> Ask the Gen AI author (based on their traits) to respond to the simulated scenario -> Simulated Gen AI person response